# Import

In [ ]:
import numpy as np

# Define function

In [ ]:
def f(x):
    return np.exp(-2*x)*np.cos(10*x)

int_range = [0,np.pi]
tol = 1.0e-6

# Define Answer (to check off of)

In [ ]:
def int_f(x):
    return (5/52)*(np.exp(-2*x))*(np.sin(10*x)) - (1/52)*(np.exp(-2*x))*(np.cos(10*x))

int_actual = int_f(int_range[1]) - int_f(int_range[0])

# Make function to check number of iterattions needed to get within a tolerance

In [ ]:
def check_integration_method(method,f,a,b,N,tolerance):
    for i in range(len(N)):
        int_test = method(f,a,b,N[i])
        if np.abs(int_actual-int_test)<=tolerance:
            print('Accuracy reached at N = {}'.format(N[i]))
            return N[i]
    print('Specified Accuracy not reached with given array of N')

# Trapezoid Method

In [ ]:
def trapezoid_core(f,x,h):
    # f=function; x=x_i; h=( x_i+1 - x_i )
    return 0.5*h*(f(x+h)+f(x))

def trapezoid_method(f,a,b,N):
    # f=function; a,b=limits of integration; N=#of evaluations
    x = np.linspace(a,b,N)    # make x go from a to b with array length N
    h = x[1]-x[0]             # bc of linspace, spacing is constant
    
    F_int = 0.0               # initialize value of intergral
    
    for i in range(N-1):      # perform integral using trapezoid method
        F_int += trapezoid_core(f,x[i],h)
    return F_int

# Simpson's Method

In [ ]:
def simpson_core(f,x,h):
    # f=function; x=x_i; h=( x_i+1 - x_i )
    return h*(f(x) + 4*f(x+h) + f(x+2*h))/3

def simpson_method(f,a,b,N):
    # f=function; a,b=limits of integration; N=#of evaluations
    x = np.linspace(a,b,N)    # make x go from a to b with array length N
    h = x[1]-x[0]             # bc of linspace, spacing is constant
    
    F_int = 0.0               # initialize value of intergral
    
    for i in range(0,N-2,2):  # perform integral using simpson method
        F_int += simpson_core(f,x[i],h)
    if ((N%2)==0):            # since simpsons method uses the next two terms, this is necesarry
        F_int += simpson_core(f,x[-2],0.5*h)
    return F_int

# Romberg integration

In [ ]:
def romberg_core(f,a,b,i):
    h = b-a          # difference of a and b
    
    dh = h/2**(i)
    
    K = h/2**(i+1)
    
    M = 0.0
    for j in range(2**i):
        M += f(a + 0.5*dh + j*dh)
    
    return K*M

def romberg_integration(f,a,b,tol,quiet=False):
    i = 0
    imax=1000
    
    delta = 100.0*np.fabs(tol)
    
    I = np.zeros(imax,dtype=float)
    
    I[0] = 0.5*(b-a)*(f(a) + f(b))
    
    i+=1
    
    if not quiet:
        print('{0:^16}{1:^25}{2:^25}{3:^25}'                                   # header for printed table
          .format('Iteration','I[i]','I[i-1]','delta'))
    
    while(delta>tol):
            I[i] = 0.5*I[i-1] + romberg_core(f,a,b,i)
            
            delta = np.fabs( (I[i] - I[i-1])/I[i] )
            
            if not quiet:
                print('{0:^16}{1:^25}{2:^25}{3:^25}' .format(i,I[i],I[i-1],delta))
            
            if delta>tol:
                i += 1
                if i>imax:
                    print('Max iterations reached.')
                    raise StopIteration('Stopping iterations after ',i)
    print('Using Rombergs method, Integral of f(x) from {} to {:.2f} = {}'.format(int_range[0],int_range[1],I[i]))
    print('     This answer was found in {} iterations to be within {:.2e} of the actual answer'.format(i,tol))
    return I[i]

# Running Each method

In [ ]:
# Running Trapezoid Method
N_array = np.arange(100,1500)   # array of N values to check. I guessed the correct N value is bewteen 100 and 1500
                                # if this guess is wrong, the check_integration_method code will mention that
N = check_integration_method(trapezoid_method,f,int_range[0],int_range[1],N_array,tol)

int_trapezoid = trapezoid_method(f,int_range[0],int_range[1],N)

print('Using Trapezoid Method with {} intervals:'.format(N))
print('     Integral of f(x) from {} to {:.2f} = {}'
     .format(int_range[0],int_range[1],int_trapezoid))
print()
print('Analytic answer: {}'.format(int_actual))
print('     Difference: {}'.format(np.abs(int_actual-int_trapezoid)))

### Number of intervals needed to reach specified accuracy using Trapezoid Method: 1283

In [ ]:
# Running Simpson Method
N_array = np.arange(100,1000)
N = check_integration_method(simpson_method,f,int_range[0],int_range[1],N_array,tol)

int_simpson = simpson_method(f,int_range[0],int_range[1],N)

print('Using Simpson Method with {} intervals:'.format(N))
print('     Integral of f(x) from {} to {:.2f} = {}'
     .format(int_range[0],int_range[1],int_simpson))
print()
print('Analytic answer: {}'.format(int_actual))
print('     Difference: {}'.format(np.abs(int_actual-int_simpson)))

### Number of intervals needed to reach specified accuracy using Simpson's Method:  136

In [ ]:
int_romberg = romberg_integration(f,int_range[0],int_range[1],tol)

### Number of itterations used in Romberg integration: 26